<a href="https://colab.research.google.com/github/tuntul17/NBA-Analysis/blob/main/NBA_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#operation sys.
import os,shutil
#as ususal
import pandas as pd
import numpy as np
#to scrape
import requests as rq
import time #Just curious how will it take
#plot
from pylab import *
import plotly #this is new package for me
import seaborn as sns #this is also new
#ML
import sklearn as sk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import time

# Web scrapping:

In [ ]:
#before all the web scraping, I want to create a base df!
url = "https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=Rookies&Season=2020-21&SeasonType=Playoffs&StatCategory=PTS"
R = rq.get(url).json()
headers = R["resultSet"]["headers"]

In [ ]:
player = ["S","Rookies"]
year = np.arange(2000,2022)
season_type = ["Playoffs","Regular%20Season"]
df = pd.DataFrame(columns=headers)
T = time.time()
for y in year:
  for s in season_type:
    for p in player:
      if y-2000<10:
        url = f"https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope={p}&Season={str(y)}-0{str(y-1999)}&SeasonType={s}&StatCategory=PTS"
      if y-2000>=10:
        url = f"https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope={p}&Season={str(y)}-{str(y-1999)}&SeasonType={s}&StatCategory=PTS"
      R = rq.get(url).json()
      #df = pd.DataFrame(R["resultSet"]['rowSet'],columns = headers)
      temp_df1 = pd.DataFrame(R["resultSet"]["rowSet"],columns= headers)
      temp_df2 = pd.DataFrame({'Year':[int(y) for i in range(len(temp_df1))],
                      "Rookie":[p for i in range(len(temp_df1))],
                      'Season_type':[s for i in range(len(temp_df1))]})
      temp3 = pd.concat([temp_df2,temp_df1],axis=1)
      df = pd.concat([temp3,df],axis=0)
      
df['Season_type'].replace("Regular%20Season","Regular Season",inplace=True)
df["Rookie"].replace("S","Non-Rookie",inplace=True)
df["Rookie"].replace("Rookies","Rookie",inplace=True)
print(f"web scraping took {round(time.time()-T,2)} second ")
T1 = time.time()
df.to_excel("RAW_20_NBA.xlsx",index=False)
print(f"saving as a xlsx file took {round(time.time()-T1,2)} second")

web scraping took 6.85 second 
saving as a xlsx file took 11.91 second


* **This is raw data**
* Move this file to drive just in case!

In [ ]:
shutil.copy("/content/RAW_20_NBA.xlsx","/content/drive/MyDrive/Colab Notebooks")

'/content/drive/MyDrive/Colab Notebooks/RAW_20_NBA.xlsx'

# Data Cleaning:

In [ ]:
rawData = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/RAW_20_NBA.xlsx")
#rawData = pd.read_excel("/content/RAW_20_NBA.xlsx")

In [ ]:
#rawData.sample(5)
rawData.sample(5)

,Year,Rookie,Season_type,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
6996,2013,Non-Rookie,Regular Season,1713,89,Vince Carter,1610612742,DAL,81,1973,...,284,212,61,35,108,209,967,935,1.96,0.56
4566,2016,Non-Rookie,Regular Season,201609,28,Goran Dragic,1610612748,MIA,73,2459,...,279,423,89,13,212,199,1483,1406,2.00,0.42
14870,2002,Non-Rookie,Playoffs,353,84,Darrell Armstrong,1610612753,ORL,7,226,...,17,26,6,0,17,22,66,72,1.53,0.35
13924,2003,Non-Rookie,Regular Season,1738,293,Ansu Sesay,1610612760,SEA,57,583,...,92,19,19,20,23,66,200,216,0.83,0.83
12830,2004,Rookie,Regular Season,2730,3,Dwight Howard,1610612753,ORL,82,2671,...,823,75,77,136,165,232,981,1466,0.46,0.47


In [ ]:
rawData.drop(columns=["RANK","TEAM_ID","PLAYER_ID"],inplace=True)
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16398 entries, 0 to 16397
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         16398 non-null  int64  
 1   Rookie       16398 non-null  object 
 2   Season_type  16398 non-null  object 
 3   PLAYER       16398 non-null  object 
 4   TEAM         16398 non-null  object 
 5   GP           16398 non-null  int64  
 6   MIN          16398 non-null  int64  
 7   FGM          16398 non-null  int64  
 8   FGA          16398 non-null  int64  
 9   FG_PCT       16398 non-null  float64
 10  FG3M         16398 non-null  int64  
 11  FG3A         16398 non-null  int64  
 12  FG3_PCT      16398 non-null  float64
 13  FTM          16398 non-null  int64  
 14  FTA          16398 non-null  int64  
 15  FT_PCT       16398 non-null  float64
 16  OREB         16398 non-null  int64  
 17  DREB         16398 non-null  int64  
 18  REB          16398 non-null  int64  
 19  AST 

In [ ]:
rawData.TEAM.unique()

array(['ORL', 'HOU', 'TOR', 'DET', 'CLE', 'SAC', 'NOP', 'IND', 'DEN',
       'CHI', 'OKC', 'GSW', 'WAS', 'BKN', 'MEM', 'LAL', 'POR', 'LAC',
       'MIA', 'SAS', 'NYK', 'UTA', 'MIL', 'CHA', 'PHX', 'PHI', 'BOS',
       'ATL', 'MIN', 'DAL', 'NOH', 'NJN', 'SEA', 'NOK', 'CHH', 'VAN'],
      dtype=object)

In [ ]:
rawData['TEAM'].replace(to_replace=["NOP","NOH","NOK"],value = "NOP", inplace = True) #New Orleans Hornets will be replaced w/ New Orleans Pelicans (Really?)
rawData['TEAM'].replace(to_replace=["BKN","NJN"],value = "BKN", inplace = True) #New Jersey Nets will be replaced w/ Brooklyn Nets
rawData["TEAM"].replace(to_replace=["SEA"],value = "OKC",inplace = True) #Seattle Supersonics ==> Oklahama City Thunder in '08
rawData["TEAM"].replace(to_replace=["VAN"],value = "MEM",inplace = True) #Vancouver Grizzlies ==> Memphis Grizzlies in '01
rawData["TEAM"].replace(to_replace=["CHH"],value = "CHA",inplace = True) #Charlotte Bobcats to Charlotte Hornets

Seperate Playoff & Regular Season data for later.

In [ ]:
playoff_data = rawData[rawData["Season_type"]=="Playoffs"]
Regular_data = rawData[rawData["Season_type"]=="Reggular Season"]

In [ ]:
rawData.columns

Index(['Year', 'Rookie', 'Season_type', 'PLAYER', 'TEAM', 'GP', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'EFF',
       'AST_TOV', 'STL_TOV'],
      dtype='object')

In [ ]:
#important columns:
col = ["GP","MIN","FGM","FGA","FG3M","FG3A","FTM","FTA","OREB","DREB","REB","AST","STL","BLK","TOV","PF","PTS"]
#the other columns will create a chaos as they are percentage stats

In [ ]:
#groupby 
rok = rawData.groupby(["Rookie","PLAYER","Year"])[col].sum().reset_index()
rok.sample(10)

,Rookie,PLAYER,Year,GP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
8316,Non-Rookie,Royal Ivey,2004,62,809,85,198,3,9,47,67,19,65,84,103,39,7,58,140,220
9183,Non-Rookie,Thomas Gardner,2008,19,117,14,49,7,29,3,5,0,9,9,4,5,1,3,8,38
2014,Non-Rookie,Damon Jones,2004,97,3074,393,855,264,612,86,116,17,255,272,410,51,5,115,145,1136
7876,Non-Rookie,Ramon Sessions,2014,74,1406,151,404,29,85,146,192,25,143,168,205,33,1,96,87,477
1185,Non-Rookie,Bruce Bowen,2003,92,2922,234,565,88,241,69,127,51,231,282,123,88,36,94,190,625
2116,Non-Rookie,Danny Granger,2011,73,2482,460,1115,149,396,277,319,94,275,369,136,68,44,139,175,1346
3284,Non-Rookie,Fabricio Oberto,2010,5,45,3,5,0,0,1,2,3,4,7,0,0,0,3,10,7
7102,Non-Rookie,Mirza Teletovic,2012,54,500,71,185,37,108,9,11,33,62,95,23,13,8,20,53,188
5522,Non-Rookie,Kendrick Perkins,2017,1,15,1,2,0,0,1,2,0,1,1,2,1,0,1,1,3
4514,Non-Rookie,Jason Williams,2001,65,2238,376,985,127,430,80,101,22,173,195,519,111,7,214,102,959


In [ ]:
rok.drop_duplicates(subset=rok.columns[1:],keep="last",inplace=True)
#This step is important because rawdataset includes both label in players rookie year

In [ ]:
rok = rok[rok["GP"]>=5]

In [ ]:
rok[rok["Rookie"]=="Rookie"].sample(10)

,Rookie,PLAYER,Year,GP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
10699,Rookie,Isaiah Canaan,2013,22,252,31,87,18,55,21,29,7,17,24,22,8,4,21,25,101
10345,Rookie,Chris Kaman,2003,82,1843,200,435,0,2,99,142,126,335,461,85,23,73,155,211,499
10503,Rookie,Devin Harris,2004,85,1251,164,382,46,137,84,112,28,85,113,180,81,20,92,144,458
10727,Rookie,JaKarr Sampson,2014,74,1131,146,346,31,127,63,94,35,128,163,77,38,26,76,135,386
11023,Rookie,Kirk Snyder,2004,68,905,122,328,30,85,64,96,50,71,121,36,26,19,58,116,338
11136,Rookie,Marcus Derrickson,2018,11,67,16,33,10,20,4,5,8,5,13,1,0,1,3,9,46
11017,Rookie,Killian Tillie,2020,18,182,19,57,10,33,9,11,6,18,24,8,5,8,2,20,57
11134,Rookie,Marcus Banks,2003,85,1443,184,458,29,91,103,135,31,109,140,182,90,14,128,180,500
10815,Rookie,Jawad Williams,2008,10,20,5,12,2,6,0,0,1,1,2,0,1,0,1,2,12
11200,Rookie,Michael Carter-Williams,2013,70,2414,427,1054,55,208,258,367,101,336,437,441,130,43,247,213,1167


In [ ]:
"""#rok per min:
for i in rok.columns[4:]:
  rok[i] = rok[i]/rok["GP"]

rok.sample(10)"""

'#rok per min:\nfor i in rok.columns[4:]:\n  rok[i] = rok[i]/rok["GP"]\n\nrok.sample(10)'

In [ ]:
#now add back to percentages to the 
rok["FG%"] = rok["FGM"]/rok["FGA"]
rok["3PT%"] = rok["FG3M"]/rok["FG3A"]
rok["FT%"] = rok["FTM"]/rok["FTA"]
rok["FG3A%"] = rok["FG3A"]/rok["FGA"]

rok["PTS/FGA"] = rok["PTS"]/rok["FGA"]
rok["FG3M/FGM"] = rok["FG3M"]/rok["FGM"]
rok["FTA/FGA"] = rok["FTA"]/rok["FGA"]
rok["TRU%"] = 0.5*rok["PTS"]/(rok["FGA"]+0.475*rok["FTA"])
rok["AST/TOV"] = rok["AST"]/rok["TOV"]
rok.sample(10)

,Rookie,PLAYER,Year,GP,MIN,FGM,FGA,FG3M,FG3A,FTM,...,PTS,FG%,3PT%,FT%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST/TOV
407,Non-Rookie,Andre Roberson,2020,5,63,2,14,1,8,1,...,6,0.142857,0.125000,0.500000,0.571429,0.428571,0.500000,0.142857,0.200669,2.000000
8154,Non-Rookie,Robert Horry,2006,86,1485,115,311,63,186,52,...,345,0.369775,0.338710,0.641975,0.598071,1.109325,0.547826,0.260450,0.493598,1.827586
3749,Non-Rookie,Henry Ellenson,2018,19,259,39,95,17,38,19,...,114,0.410526,0.447368,0.760000,0.400000,1.200000,0.435897,0.263158,0.533333,1.777778
1287,Non-Rookie,Calbert Cheaney,2002,86,2473,335,678,10,25,42,...,722,0.494100,0.400000,0.575342,0.036873,1.064897,0.029851,0.107670,0.506542,1.486957
7733,Non-Rookie,Phil Pressey,2014,52,605,64,177,16,66,33,...,177,0.361582,0.242424,0.673469,0.372881,1.000000,0.250000,0.276836,0.441892,2.853659
5493,Non-Rookie,Kemba Walker,2017,80,2736,588,1363,231,601,363,...,1770,0.431401,0.384359,0.864286,0.440939,1.298606,0.392857,0.308144,0.566400,2.480447
159,Non-Rookie,Alan Anderson,2013,90,2034,219,547,93,279,104,...,635,0.400366,0.333333,0.764706,0.510055,1.160878,0.424658,0.248629,0.519130,1.333333
8627,Non-Rookie,Seth Curry,2020,69,2019,343,699,167,361,84,...,937,0.490701,0.462604,0.875000,0.516452,1.340486,0.486880,0.137339,0.629197,2.440000
8053,Non-Rookie,Reggie Williams,2016,6,79,8,23,5,11,6,...,27,0.347826,0.454545,0.857143,0.478261,1.173913,0.625000,0.304348,0.512821,inf
4804,Non-Rookie,Joakim Noah,2017,7,40,5,10,0,0,2,...,12,0.500000,NaN,0.500000,0.000000,1.200000,0.000000,0.400000,0.504202,1.000000


# Guess if a player is a rookie:

In [ ]:
rok.fillna(0,inplace=True)
rok.sample(10)

,Rookie,PLAYER,Year,GP,MIN,FGM,FGA,FG3M,FG3A,FTM,...,PTS,FG%,3PT%,FT%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST/TOV
2905,Non-Rookie,Dwight Howard,2019,87,1588,241,334,4,7,135,...,621,0.721557,0.571429,0.521236,0.020958,1.859281,0.016598,0.775449,0.679394,0.540000
3431,Non-Rookie,Gary Payton II,2021,83,1451,241,388,51,135,47,...,580,0.621134,0.377778,0.618421,0.347938,1.494845,0.211618,0.195876,0.683801,1.739130
5994,Non-Rookie,Landry Shamet,2020,73,1609,204,497,139,359,70,...,617,0.410463,0.387187,0.843373,0.722334,1.241449,0.681373,0.167002,0.575104,1.962963
1906,Non-Rookie,D.J. Augustin,2019,62,1548,202,507,78,218,183,...,665,0.398422,0.357798,0.897059,0.429980,1.311637,0.386139,0.402367,0.550588,2.979592
4740,Non-Rookie,Jerry Stackhouse,2001,86,3047,577,1484,104,353,547,...,1805,0.388814,0.294618,0.854688,0.237871,1.216307,0.180243,0.431267,0.504754,1.522184
713,Non-Rookie,Austin Croshere,2005,56,1323,144,323,63,163,106,...,457,0.445820,0.386503,0.883333,0.504644,1.414861,0.437500,0.371517,0.601316,1.277778
2693,Non-Rookie,Dikembe Mutombo,2005,64,955,50,95,0,0,69,...,169,0.526316,0.000000,0.758242,0.000000,1.778947,0.000000,0.957895,0.611322,0.114286
11682,Rookie,Xavier Tillman,2020,62,1103,168,304,22,66,34,...,392,0.552632,0.333333,0.641509,0.217105,1.289474,0.130952,0.174342,0.595428,1.652174
5445,Non-Rookie,Keith McLeod,2006,48,718,73,188,16,45,69,...,231,0.388298,0.355556,0.884615,0.239362,1.228723,0.219178,0.414894,0.513219,1.956522
10448,Rookie,David Noel,2006,68,792,65,177,17,53,37,...,184,0.367232,0.320755,0.860465,0.299435,1.039548,0.261538,0.242938,0.466000,1.625000


Prepare the train and test data:

In [ ]:
Y = np.array(rok["Rookie"])
#this may be stupid: (Rookie ==1, Non-Rookie == 0)
for i in range(len(Y)):
  if Y[i] == "Rookie":
    Y[i] =1
  else:
    Y[i] = 0

X = np.array(rok[rok.columns[3:]])
where_are_NaNs = isnan(X)
X[where_are_NaNs] = 0
print(X.shape)

(9671, 26)


In [ ]:
xtr,xts,ytr,yts = train_test_split(X,Y,train_size = 0.8,shuffle=True,random_state=42)
xtr=xtr.astype('int')
xts=xts.astype('int')
ytr = ytr.astype("int")
yts = yts.astype('int')

In [ ]:
#knn model:
knn = KNeighborsClassifier()

In [ ]:
#fit the model:
knn.fit(xtr,ytr)

KNeighborsClassifier()

In [ ]:
y_pred = knn.predict(xts)

In [ ]:
lab = ["Non-Rookie","Rookie"]
Report = classification_report(yts,y_pred,target_names = lab) 
acc = sk.metrics.accuracy_score(yts,y_pred,normalize=False)
print("total test value: ",len(yts))
print("total correct ans: ", acc)
print(Report)

total test value:  1935
total correct ans:  1607
              precision    recall  f1-score   support

  Non-Rookie       0.86      0.95      0.91      1651
      Rookie       0.30      0.12      0.17       284

    accuracy                           0.83      1935
   macro avg       0.58      0.53      0.54      1935
weighted avg       0.78      0.83      0.80      1935



Without any optimization knn can not differentiate rookies
but it can find non-rookies
> Time to optimize a little bit


In [ ]:
#First get the hyperparameters
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [ ]:
#this may take a lot of time.
from sklearn.model_selection import GridSearchCV
T = time.time()
#hyperparameter range:
n_neighbors = list(range(21,41,2)) 
weights = ["uniform","distance"]
algorithm = ["auto","ball_tree","brute"]
leaf_size = list(range(1,11))
p = [1,2]

hyperparameters= dict(leaf_size=leaf_size,n_neighbors= n_neighbors,algorithm= algorithm,weights=weights,p=p)

#Create a second model to optimize:
knn_2 = KNeighborsClassifier()

clf = GridSearchCV(knn_2,hyperparameters,cv=2)
#use optimum parameters to get best result for fitting
best_model = clf.fit(xtr,ytr)
End_time = time.time()
#print the optimum parameters:
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])
print("Best weights: ",best_model.best_estimator_.get_params()["weights"])
print("Best algorithm: ",best_model.best_estimator_.get_params()["algorithm"])

#predict:
y_pred2 = best_model.predict(xts)

report2 = classification_report(yts,y_pred2,target_names=lab)


Best leaf_size: 1
Best p: 2
Best n_neighbors: 33
Best weights:  distance
Best algorithm:  auto
Total time: 954.0519909858704



In [ ]:
print(f"Total time: {round((End_time-T)/60,2)} min")
print(report2)
print(confusion_matrix(yts,y_pred2))

Total time: 15.9 min
              precision    recall  f1-score   support

  Non-Rookie       0.86      0.99      0.92      1651
      Rookie       0.45      0.05      0.08       284

    accuracy                           0.85      1935
   macro avg       0.65      0.52      0.50      1935
weighted avg       0.80      0.85      0.80      1935

[[1635   16]
 [ 271   13]]


# Data Visualisation:

,Year,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
Year,1.000000,-0.083342,-0.083974,-0.033341,-0.043796,0.040741,0.137608,0.143395,0.153240,-0.086117,...,-0.055791,-0.026289,-0.071843,-0.062368,-0.088024,-0.129530,-0.028621,-0.024349,0.101728,0.057129
GP,-0.083342,1.000000,0.891914,0.778456,0.785663,0.219868,0.558961,0.579935,0.195310,0.652411,...,0.760104,0.629196,0.779859,0.550665,0.770653,0.897993,0.767527,0.798194,0.152270,0.081007
MIN,-0.083974,0.891914,1.000000,0.938642,0.943514,0.206591,0.666123,0.685753,0.228204,0.828305,...,0.841346,0.777669,0.892188,0.587760,0.908430,0.910169,0.933471,0.943956,0.179677,0.043393
FGM,-0.033341,0.778456,0.938642,1.000000,0.991006,0.212658,0.663427,0.681160,0.223665,0.904829,...,0.815209,0.771606,0.836028,0.559929,0.921499,0.820408,0.994486,0.965526,0.141665,-0.015876
FGA,-0.043796,0.785663,0.943514,0.991006,1.000000,0.171125,0.712885,0.735616,0.250554,0.897067,...,0.775840,0.793798,0.851605,0.510595,0.926952,0.812231,0.991304,0.942772,0.163605,-0.008451
FG_PCT,0.040741,0.219868,0.206591,0.212658,0.171125,1.000000,0.035561,0.025501,0.124331,0.176908,...,0.273534,0.100447,0.154423,0.252888,0.175568,0.247615,0.197384,0.249082,0.009920,0.064526
FG3M,0.137608,0.558961,0.666123,0.663427,0.712885,0.035561,1.000000,0.991884,0.417906,0.530979,...,0.351775,0.617748,0.625043,0.121846,0.593830,0.487656,0.702503,0.599312,0.238888,0.068908
FG3A,0.143395,0.579935,0.685753,0.681160,0.735616,0.025501,0.991884,1.000000,0.408343,0.553970,...,0.368230,0.644461,0.652870,0.132939,0.622974,0.506021,0.719571,0.614618,0.246984,0.071164
FG3_PCT,0.153240,0.195310,0.228204,0.223665,0.250554,0.124331,0.417906,0.408343,1.000000,0.160168,...,0.041321,0.255780,0.238136,-0.067376,0.194930,0.119617,0.241133,0.181488,0.298517,0.136287
FTM,-0.086117,0.652411,0.828305,0.904829,0.897067,0.176908,0.530979,0.553970,0.160168,1.000000,...,0.733623,0.724474,0.756023,0.509928,0.883945,0.721998,0.931099,0.894307,0.099383,-0.051868
